In [ ]:
# import libraries
import os
import time
import glob
import cloudscraper
from lxml import html, etree
import tqdm
import json

# download sitemap.xml

In [ ]:
scraper = cloudscraper.create_scraper()
sitemap_url = "https://animalia.bio/sitemap.xml"
sitemap_response = scraper.get(sitemap_url)
if sitemap_response.status_code == 200 and "<loc>" in sitemap_response.text:
    os.makedirs(f'raw-data', exist_ok=True)
    with open("raw-data/sitemap.xml", "w", encoding="utf-8") as file:
        file.write(sitemap_response.text)
        print(sitemap_response.text)
else:
    print("blocked by Cloudflare")

# download xmls according to the sitemap.xml

In [ ]:
with open('raw-data/sitemap.xml', 'r') as file:
    content = ''.join(file.readlines())
tree = html.fromstring(content.encode('utf-8'))
urls = tree.xpath("//loc/text()")

for url in urls:
    response = scraper.get(url)
    if response.status_code == 200:
        file_name = url.split('/')[-1]
        os.makedirs(f'raw-data/{file_name.split(".")[0]}', exist_ok=True)
        with open(f'raw-data/{file_name.split(".")[0]}/{file_name}', "w", encoding="utf-8") as file:
            file.write(response.text)
        print(f'raw-data/{file_name.split(".")[0]}/{file_name}')
    else:
        print("blocked by Cloudflare")

## download the image-text pairs from the urls in collections.xml

In [ ]:
subsets = [item.split('/')[-1] for item in glob.glob('raw-data/*') if not item.endswith('.xml')]
print(subsets)
for subset in subsets:
    with open(f'raw-data/{subset}/{subset}.xml', 'r') as file:
        content = ''.join(file.readlines())
    tree = html.fromstring(content.encode('utf-8'))
    urls = tree.xpath("//loc/text()")

    scraper = cloudscraper.create_scraper()
    for url in urls:
        response = scraper.get(url)
        if response.status_code == 200:
            file_name = url.split('/')[-1]
            if not file_name.endswith('.xml'):
                file_name += '.xml'
            os.makedirs(f'raw-data/{subset}/{file_name.split(".")[0]}', exist_ok=True)
            with open(f'raw-data/{subset}/{file_name.split(".")[0]}/{file_name}', "w", encoding="utf-8") as file:
                file.write(response.text)
                print(f'raw-data/{subset}/{file_name.split(".")[0]}/{file_name}')
                time.sleep(1)
        else:
            print("blocked by Cloudflare")


In [ ]:

subsets = [item.split('\\')[-1] for item in glob.glob('animals/*')]
for subset in tqdm.tqdm(subsets):
    if os.path.exists(f'animals/{subsubset}/descriptions.json'):
        continue

    with open(f'animals/{subset}/{subset}.xml', 'r', encoding='utf-8') as file:
        content = ''.join(file.readlines())
        tree = html.fromstring(content.encode('utf-8'))

        # extract urls
        img_urls = []
        img_urls += tree.xpath('//div[@class="s-char-img open-gallery"]/img/@src')
        img_urls += tree.xpath('//div[@class="s-gallery-item s-gallery-item--right open-gallery"]/img/@src')
        for img_url in img_urls:
            scraper = cloudscraper.create_scraper()
            response = scraper.get(img_url)
            if response.status_code == 200:
                file_name = os.path.basename(img_url).split("?")[0]
                file_path = f'animals/{subset}/{file_name}'
                with open(file_path, "wb") as f:
                    f.write(response.content)
            time.sleep(1)

        # extract general descriptions
        general_descriptions = tree.xpath('//div[@class="s-char-text"]/p//text()')
        general_descriptions = ''.join(general_descriptions).strip()

        # extract appearance descriptions
        appearance_descriptions = tree.xpath('//div[@class="s-appearance-block"]/p//text()')
        appearance_descriptions = ''.join(appearance_descriptions).strip()

        # extract distribution descriptions
        distribution_descriptions = tree.xpath('//div[@class="s-distr-content"]/p//text()')
        distribution_descriptions = ''.join(distribution_descriptions).strip()

        # extract lifestyle descriptions
        lifestyle_descriptions = tree.xpath('//div[@class="s-habbit-content"]/p//text()')
        lifestyle_descriptions = ''.join(lifestyle_descriptions).strip()

        # extract diet descriptions
        diet_descriptions = tree.xpath('//div[@class="s-diet-block"]/div[@class="row"]/div[@class="col-lg-8"]/p//text()')
        diet_descriptions = ''.join(diet_descriptions).strip()

        # extract mating descriptions
        mating_descriptions = tree.xpath('//div[@class="s-mating-text"]/p//text()')
        mating_descriptions = ''.join(mating_descriptions).strip()

        # extract population descriptions
        population_descriptions = tree.xpath('//div[@class="s-population-content"]/p//text()')
        population_descriptions = ''.join(population_descriptions).strip()

        # gather all information and save to json file
        descriptions = {
            'general_descriptions': general_descriptions,
            'appearance_descriptions': appearance_descriptions,
            'distribution_descriptions': distribution_descriptions,
            'lifestyle_descriptions': lifestyle_descriptions,
            'diet_descriptions': diet_descriptions,
            'mating_descriptions': mating_descriptions,
            'population_descriptions': population_descriptions
        }
        with open(f'animals/{subset}/descriptions.json', 'w', encoding='utf-8') as file:
            json.dump(descriptions, file, indent=4)
    
